In [1]:
import tensorflow as tf
import keras
from keras import Model
from keras import layers

2024-09-23 23:19:00.070995: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-23 23:19:00.140068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 23:19:00.168956: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 23:19:00.177659: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-23 23:19:00.224048: I tensorflow/core/platform/cpu_feature_guar


#------------------------------------------------------------------------------------#
##  THIS IS A TRAINING MODEL, IMPLEMENTION DETAIL MAY DIFFER FROM PRODUCTION MODEL  ##
#------------------------------------------------------------------------------------#


# TODO
# THIS IS AN ABSTRACT BACKBONE SCRIPT, ADD ACTUAL IMPLEMENTATION LATER

# NOTE
# model detail

In [ ]:
# global variables
model = None # public

In [ ]:
# custom layer definition
class Conv2Plus1D(layers.Layer):
    def __init__(self, kernel_size, filters = 1, strides = (1,1,1), padding = 'valid'):
        """kernel_size is depth width height"""
        super().__init__()
        wh_stride = (1, strides[1], strides[2])
        t_stride = (strides[0], 1, 1)
        self.seq = keras.Sequential([  
        # Spatial decomposition
        layers.Conv3D(filters=filters,
                      kernel_size=(1, kernel_size[1], kernel_size[2]), strides = wh_stride,
                      padding=padding),
        # Temporal decomposition
        layers.Conv3D(filters=filters, 
                      kernel_size=(kernel_size[0], 1, 1), strides = t_stride,
                      padding=padding)
        ])

    def call(self, x):
        return self.seq(x)

In [ ]:
# hand model
class HandModel(Model):
    def __init__(self):
        super().__init__()
        self.conv21 = Conv2Plus1D(filters = 1, kernal_size = (3,3,1), strides = (1,1,2))
        pass

    def call(self, x):
        x = Conv2Plus1D(filters = 1, kernal_size = (3,3,1), strides = 2)
        pass

# pose model
class PoseModel(Model):
    def __init__(self):
        super().__init__()
        pass

    def call(self, inputs):
        # inputs 0: L, 1: R, 2: Pose
        
        pass


# the main model class
class SLRModel(Model):
    def __init__(self):
        super().__init__()
        pass

    def call(self, inputs):
        # inputs 0: L, 1: R, 2: Pose
        
        pass


# the part that doesnt get executed when imported
if __name__ == "__main__":
    pass


def get_model():
    return model

def set_model(new_model):
    global model
    model = new_model

def train_or_fit():
    pass